In [ ]:
!unzip '/content/drive/MyDrive/emotion-recognition/dataset/emotion detection.zip' -d './data'

In [ ]:
import torch
from torch import nn
from torchvision.models.resnet import resnet18, ResNet18_Weights
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms.v2 as T
from tqdm import tqdm
from torch import optim
from torch.nn import functional as F

## Prepare Dataloader

In [ ]:
transforms = ResNet18_Weights.IMAGENET1K_V1.transforms()
transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
train_dir = '/content/data/train'
test_dir = '/content/data/valid'

# For train
train_dataset = ImageFolder(train_dir, transforms)
train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    pin_memory=True,
    persistent_workers=True,
    num_workers=2
)

# For test
test_dataset = ImageFolder(test_dir, transforms)
test_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    pin_memory=True,
    persistent_workers=True,
    num_workers=1
)

## Prepare Model

In [ ]:
class ResNet(nn.Module):
  def __init__(self, num_classes):
    super().__init__()

    self.model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

    ## Freeze all the layer
    # for param in list(self.model.parameters())[:20]:
    #   param.requires_grad = False

    ## Modify the classification layer
    in_features = self.model.fc.in_features
    self.model.fc = nn.Linear(in_features, num_classes)

  def forward(self, x):
    x = self.model(x)
    return x

## Loss function and optimizers

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
train_dataset.classes

['angry',
 'confused',
 'disgust',
 'fear',
 'happy',
 'neutral',
 'sad',
 'shy',
 'surprise']

In [ ]:
num_classes = len(train_dataset.classes)
model = ResNet(num_classes).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 160MB/s]


In [ ]:
# Loss function & optimizer
criterion = nn.CrossEntropyLoss()
params = [param for param in model.parameters() if param.requires_grad]
optimizer = optim.Adam(params, lr=0.001)

## Training Loop

In [ ]:
epochs = 30
min_loss = float('inf')

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct_t, total_t = 0, 0

  for images, targets in tqdm(train_loader, desc=f'Training [{epoch + 1}/{epochs}]: '):
    images, targets = images.to(device), targets.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    losses = criterion(outputs, targets)
    losses.backward()
    optimizer.step()

    pred = torch.argmax(outputs, dim=1)
    correct_t += (targets == pred).sum().item()
    total_t += len(train_loader)
    running_loss += losses.item()
  t_loss = running_loss / targets.size(0)
  t_acc = 100 * correct_t / total_t
  print(f'Train losses: {t_loss:.5f}, Train accuracy: {t_acc:.2f}%')

  # Save best model
  if (min_loss > t_loss):
    best_model = f'/content/drive/MyDrive/emotion-recognition/models/resnet-best-full.pth'
    torch.save({
        'classes': train_dataset.classes,
        'model_state_dict': model.state_dict(),
        'epochs': epochs
    }, best_model)
    min_loss = t_loss

  ## Validation test
  correct_e, total_e = 0, 0
  model.eval()

  for images, targets in tqdm(test_loader, desc='Validation...'):
    images, targets = images.to(device), targets.to(device)

    with torch.no_grad():
      outputs = model(images)
      predictions = torch.argmax(outputs, dim=1)
    total_e += targets.size(0)
    correct_e += sum(predictions == targets).item()
  acc = 100.0 * correct_e / total_e
  print(f'Model accuracy on testing: {acc:.2f}%\n')

model_path = f'/content/drive/MyDrive/emotion-recognition/models/resnet-e{epochs}-full.pth'
torch.save({
    'classes': train_dataset.classes,
    'model_state_dict': model.state_dict(),
    'epochs': epochs
}, model_path)
print(f'Model saved successfully at {model_path}')

Training [1/30]: 100%|██████████| 163/163 [01:19<00:00,  2.06it/s]


Train losses: 127.51695, Train accuracy: 34.42%


Validation...: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]


Model accuracy on testing: 46.58%



Training [2/30]: 100%|██████████| 163/163 [01:17<00:00,  2.09it/s]


...

Model accuracy on testing: 52.33%



Training [30/30]: 100%|██████████| 163/163 [01:15<00:00,  2.15it/s]


Train losses: 8.99919, Train accuracy: 75.57%


Validation...: 100%|██████████| 8/8 [00:05<00:00,  1.35it/s]

Model accuracy on testing: 53.59%

Model saved successfully at /content/drive/MyDrive/emotion-recognition/models/resnet-e30-full.pth


## Model evaluation

In [ ]:
correct, total = 0, 0
model.eval()

for images, targets in tqdm(test_loader, desc=f'Validation...'):
  images, targets = images.to(device), targets.to(device)

  with torch.no_grad():
    outputs = model(images)
    predictions = torch.argmax(outputs, dim=1)
  total += targets.size(0)
  correct += sum(predictions == targets).item()
acc = 100.0 * correct / total
print(f'Model accuracy on testing: {acc:.2f}')

## Load model & inference

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
model_path = '/content/drive/MyDrive/emotion-recognition/models/resnet-e30-full.pth'
# model_path = '/content/drive/MyDrive/emotion-recognition/models/resnet-best-full.pth'
checkpoint = torch.load(model_path, map_location=device)
classes = checkpoint['classes']
model_state_dict = checkpoint['model_state_dict']

model = ResNet(len(classes)).to(device)
model.load_state_dict(model_state_dict)
checkpoint['epochs']

30

In [ ]:
from PIL import Image
from torchvision.transforms import v2 as T

In [ ]:
# Infer the model
def predict(image_path, model):
  image = Image.open(image_path).convert('RGB')
  transforms = T.Compose([
      T.ToImage(),
      T.ToDtype(torch.float32, scale=True)
  ])
  image = transforms(image).to(device)

  model.eval()
  with torch.no_grad():
    outputs = model(image.unsqueeze(0))
  predictions = torch.argmax(outputs, dim=1)
  return predictions

In [ ]:
image_path = '/content/face.jpg'
results = predict(image_path, model)
classes[results.item()]

'happy'

In [ ]:
def predict_with(image_path, model, classes):
  image = Image.open(image_path).convert('RGB')
  # transforms = T.Compose([
  #     T.ToImage(),
  #     T.ToDtype(torch.float32, scale=True),
  #     T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  # ])

  transforms = ResNet18_Weights.IMAGENET1K_V1.transforms()
  image_batch = transforms(image).to(device).unsqueeze(0)

  model.eval()
  with torch.no_grad():
    outputs = model(image_batch)
  probabilities = F.softmax(outputs[0], dim=0)
  values, id = torch.topk(probabilities, 5)

  pred = {}
  for value, id in zip(values, id):
    pred[classes[id]] = value.item()
  return pred

In [ ]:
# Neutral
image_path = '/content/drive/MyDrive/emotion-recognition/assets/face.jpg'
predict_with(image_path, model, classes)

{'neutral': 0.99979168176651,
 'surprise': 0.00011066641309298575,
 'confused': 4.494014865485951e-05,
 'angry': 2.68292824330274e-05,
 'happy': 1.7674638002063148e-05}

In [ ]:
# Neutral
image_path = '/content/drive/MyDrive/emotion-recognition/assets/face1.jpg'
predict_with(image_path, model, classes)

{'neutral': 0.9999744892120361,
 'confused': 1.795478237909265e-05,
 'surprise': 6.091743671277072e-06,
 'sad': 1.3594773236036417e-06,
 'disgust': 9.24924279388506e-08}

In [ ]:
# Sad
image_path = '/content/drive/MyDrive/emotion-recognition/assets/Screenshot 2025-11-12 004551.png'
predict_with(image_path, model, classes)

{'sad': 0.6969957947731018,
 'neutral': 0.15038418769836426,
 'confused': 0.07768415659666061,
 'angry': 0.05305913835763931,
 'disgust': 0.01241245400160551}

In [ ]:
# Happy
image_path = '/content/drive/MyDrive/emotion-recognition/assets/Screenshot 2025-11-12 004909.png'
predict_with(image_path, model, classes)

{'neutral': 0.9997139573097229,
 'sad': 0.0002165273472201079,
 'surprise': 4.868244650424458e-05,
 'angry': 8.166888619598467e-06,
 'shy': 5.029511612519855e-06}